# Regime-Switching Multi-Model Trading System
Institutional Grade Architecture

In [ ]:

import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Attention, GlobalAveragePooling1D, Input
from tensorflow.keras.losses import Huber
import matplotlib.pyplot as plt


## 1. Download Data

In [ ]:

symbol = "QQQ"
df = yf.download(symbol, start="2005-01-01")

vix = yf.download("^VIX", start="2005-01-01")["Close"]
df["VIX"] = vix

df = df.dropna()


## 2. Feature Engineering

In [ ]:

df["Return"] = df["Close"].pct_change()
df["Target"] = df["Close"].shift(-5)/df["Close"] - 1

df["MA20"] = df["Close"].rolling(20).mean()
df["MA50"] = df["Close"].rolling(50).mean()
df["MA200"] = df["Close"].rolling(200).mean()

df["Volatility"] = df["Return"].rolling(14).std()

df = df.dropna()


## 3. Regime Classification

In [ ]:

conditions = [
    (df["MA50"] > df["MA200"]),
    (df["MA50"] < df["MA200"])
]

df["Regime"] = np.select(conditions, [1, -1], default=0)


## 4. Build Dataset

In [ ]:

features = ["Open","High","Low","Close","Volume","VIX","Volatility","MA20","MA50"]
data = df[features].values
y_full = df["Target"].values

window = 150

scaler = MinMaxScaler()
scaled = scaler.fit_transform(data)

X, y = [], []
for i in range(window, len(scaled)):
    X.append(scaled[i-window:i])
    y.append(y_full[i])

X = np.array(X)
y = np.array(y)

regimes = df["Regime"].values[window:]


## 5. Model Builder

In [ ]:

def build_model(input_shape):

    inputs = Input(shape=input_shape)
    x = LSTM(128, return_sequences=True)(inputs)
    x = LSTM(128, return_sequences=True)(x)
    attn = Attention()([x,x])
    x = GlobalAveragePooling1D()(attn)
    outputs = Dense(1)(x)

    model = Model(inputs, outputs)
    model.compile(optimizer="adam", loss=Huber())
    return model


## 6. Train Regime Models

In [ ]:

models = {}

for regime_value in [1, 0, -1]:
    mask = regimes == regime_value
    if mask.sum() < 200:
        continue
    model = build_model((X.shape[1], X.shape[2]))
    model.fit(X[mask], y[mask], epochs=20, batch_size=32, verbose=0)
    models[regime_value] = model

print("Trained regimes:", models.keys())


## 7. Multi-Model Prediction

In [ ]:

def predict_next():
    last = df[features].values[-window:]
    last = scaler.transform(last)
    last = last.reshape(1, window, len(features))

    regime = df["Regime"].iloc[-1]

    if regime in models:
        pred = models[regime].predict(last, verbose=0)[0,0]
    else:
        pred = 0

    return pred, regime

pred, regime = predict_next()
print("Regime:", regime)
print("Predicted 5-day return:", pred)


## 8. Position Engine

In [ ]:

position = np.clip(pred * 50, -1, 1)

if regime == -1:
    position *= 0.3

print("Position size:", position)
